# Data Warehouse Medicare Texas QA - Member Enrollment Yearly

Performing QA on tables in dw_staging before moving them to data_warehouse schema

## Initialization

Just loading packages that will be used and initializing connection to GP DB.

In [5]:
import pandas as pd
import sys
import psycopg2
from tqdm.notebook import tqdm
sys.path.append('H:/uth_helpers')
from db_utils import get_dsn

In [6]:
connection = psycopg2.connect(get_dsn())
connection.autocommit = True

## Member Counts and Member Months

This table is an aggregate of the member_enrollment_monthly table where enrollment data is aggregated to a yearly level.

Because of this, we cannot compare the row count of the raw tables with this table since enrollment tables from raw data sources are usually not on a yearly level. Instead, we can try to compare the Member Months (MM) and other columns to see if the aggregrate logic implemented is working as intended

In [7]:
query = '''drop table if exists qa_reporting.dw_mcrt_mbr_enrl_yearly;
create table qa_reporting.dw_mcrt_mbr_enrl_yearly
(
    data_source text,
    calendar_year int,
    table_src text,
    dw_row_count int,
    dw_uth_mbr_id_count int,
    dw_src_mbr_id_count int,
    src_mbr_count int,
    mbr_count_diff int,
    mbr_count_percentage float,
    dw_mm int,
    src_mm int,
    mm_diff int,
    mm_diff_percentage float,
    date_generated date
);
'''

with connection.cursor() as cursor:
    cursor.execute(query)

In [8]:
with connection.cursor() as cursor:
      query = '''
insert into qa_reporting.dw_mcrt_mbr_enrl_yearly
(data_source, calendar_year, table_src, dw_row_count, dw_uth_mbr_id_count, dw_src_mbr_id_count, dw_mm,date_generated)
select data_source, 
        year, 
        table_id_src, 
        count(*),
        count(distinct uth_member_id),
        count(distinct member_id_src),
        sum(total_enrolled_months),
        now()::date
  from dw_staging.mcrt_member_enrollment_yearly
 group by 1,2,3;
      '''

      cursor.execute(query)


In [9]:
with connection.cursor() as cursor:
    
      query = '''
update qa_reporting.dw_mcrt_mbr_enrl_yearly a
set src_mbr_count = b.pat_count,
    mbr_count_diff = a.dw_src_mbr_id_count - b.pat_count,
    mbr_count_percentage = abs(a.dw_src_mbr_id_count - b.pat_count) / b.pat_count
from qa_reporting.medicare_texas_counts b
where data_source = 'mcrt'
and a.calendar_year = b.year
and a.table_src = 'medicare_texas.' || b.table_name
;
      '''

      cursor.execute(query)

      query = '''
with enrl_months as(
select year::int, bene_id, t.month_year_id as enrolled_month
            from medicare_texas.mbsf_abcd_summary a
            cross join lateral (values (a.year || '01', a.mdcr_status_code_01), (a.year || '02', a.mdcr_status_code_02),
                              (a.year || '03', a.mdcr_status_code_03), (a.year || '04', a.mdcr_status_code_04), (a.year || '05', a.mdcr_status_code_05),
                              (a.year || '06', a.mdcr_status_code_06), (a.year || '07', a.mdcr_status_code_07), (a.year || '08', a.mdcr_status_code_08),
                              (a.year || '09', a.mdcr_status_code_09), (a.year || '10', a.mdcr_status_code_10), (a.year || '11', a.mdcr_status_code_11),
                              (a.year || '12', a.mdcr_status_code_12))
            t(month_year_id, enrollment_status)
            where t.enrollment_status in ('10','11','20','21','31')
),
enrl_my as (
    select 'medicare_texas.mbsf_abcd_summary' table_name, year, count(enrolled_month) as mm
    from enrl_months
    group by 2
)
update qa_reporting.dw_mcrt_mbr_enrl_yearly a
set src_mm = b.mm,
    mm_diff = a.dw_mm - b.mm,
    mm_diff_percentage = abs(a.dw_mm - b.mm) / b.mm
from enrl_my b
where data_source = 'mcrt'
and a.calendar_year = b.year
and a.table_src = b.table_name
;
      '''

      cursor.execute(query)

In [10]:
df = pd.read_sql('''select * 
from qa_reporting.dw_mcrt_mbr_enrl_yearly 
order by mm_diff ;''', con=connection)
df

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,data_source,calendar_year,table_src,dw_row_count,dw_uth_mbr_id_count,dw_src_mbr_id_count,src_mbr_count,mbr_count_diff,mbr_count_percentage,dw_mm,src_mm,mm_diff,mm_diff_percentage,date_generated
0,mcrt,2018,medicare_texas.mbsf_abcd_summary,4284273,4284273,4284273,4284529,-256,0.0,48871117,48871117,0,0.0,2023-12-18
1,mcrt,2021,medicare_texas.mbsf_abcd_summary,4818236,4818236,4818236,4818441,-205,0.0,54948770,54948770,0,0.0,2023-12-18
2,mcrt,2017,medicare_texas.mbsf_abcd_summary,4194036,4194036,4194036,4194289,-253,0.0,47866081,47866081,0,0.0,2023-12-18
3,mcrt,2019,medicare_texas.mbsf_abcd_summary,4419219,4419219,4419219,4419443,-224,0.0,50466984,50466984,0,0.0,2023-12-18
4,mcrt,2016,medicare_texas.mbsf_abcd_summary,4068903,4068903,4068903,4069556,-653,0.0,46448004,46448004,0,0.0,2023-12-18
5,mcrt,2015,medicare_texas.mbsf_abcd_summary,3948967,3948967,3948967,3949215,-248,0.0,45003185,45003185,0,0.0,2023-12-18
6,mcrt,2014,medicare_texas.mbsf_abcd_summary,3822495,3822495,3822495,3822796,-301,0.0,43529167,43529167,0,0.0,2023-12-18
7,mcrt,2020,medicare_texas.mbsf_abcd_summary,4727118,4727118,4727118,4727375,-257,0.0,54015078,54015078,0,0.0,2023-12-18


Here we look at the overall difference of member enrollment and member count. We start of by just adding the counts for each of the source tables. However, due to how the yearly table is built, it may be better to look at member counts and member months as the data source overall.

In [11]:
df.groupby('calendar_year')[['mm_diff', 'mm_diff_percentage', 'mbr_count_diff', 'mbr_count_percentage']].sum()

,mm_diff,mm_diff_percentage,mbr_count_diff,mbr_count_percentage
calendar_year,,,,
2014,0,0.0,-301,0.0
2015,0,0.0,-248,0.0
2016,0,0.0,-653,0.0
2017,0,0.0,-253,0.0
2018,0,0.0,-256,0.0
2019,0,0.0,-224,0.0
2020,0,0.0,-257,0.0
2021,0,0.0,-205,0.0


## Gender Counts

Just like with the monthly enrollment table, we need to check the counts and values of the other columns. Here we take a look at the gender_cd column

In [12]:
query = '''with mcrn_gen_cd as (
    select distinct year::int, bene_id, sex_ident_cd
    from medicare_texas.mbsf_abcd_summary a
),
mcrn_gen as (
    select year, c.gender_cd, count(*) gender_count
    from mcrn_gen_cd m
    left join reference_tables.ref_gender c
    on c.data_source = 'mcr'
   and c.gender_cd_src = m.sex_ident_cd
    group by 1,2
), dw_gen as (
    select year, gender_cd, count(*) gender_count
    from dw_staging.mcrt_member_enrollment_yearly
    group by 1,2
)
select a.*, b.gender_count as src_gender_count, 
        a.gender_count - b.gender_count as gender_diff, 
        100. * abs(a.gender_count - b.gender_count) / b.gender_count as gender_diff_percent
from mcrn_gen b
full outer join dw_gen a
on a.year = b.year
and a.gender_cd = b.gender_cd
order by year, gender_cd;
'''

pd.read_sql(query,  con=connection)

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,year,gender_cd,gender_count,src_gender_count,gender_diff,gender_diff_percent
0,2014,F,2068416,2068579,-163,0.007880
1,2014,M,1754078,1754216,-138,0.007867
2,2014,U,1,1,0,0.000000
3,2015,F,2134378,2134520,-142,0.006653
4,2015,M,1814588,1814694,-106,0.005841
5,2015,U,1,1,0,0.000000
6,2016,F,2197078,2197429,-351,0.015973
7,2016,M,1871823,1872125,-302,0.016131
8,2016,U,2,2,0,0.000000
9,2017,F,2262751,2262906,-155,0.006850


## Plan Type

Check if plan types are properly mapped at a yearly level. If the plan type counts do not match between the member_enrollment_yearly table and the raw enrollment table, it most likely occurred when cleaning up the plan_type column. Ideally the count difference percent should be <= 1%.

In [13]:
query = '''with mcrn_enroll as (
    select year::int, bene_id, ent.plan_type, t.month_year_id
    from medicare_texas.mbsf_abcd_summary a
    cross join lateral (values (year || '01', a.mdcr_entlmt_buyin_ind_01, a.mdcr_status_code_01), (year || '02', a.mdcr_entlmt_buyin_ind_02, a.mdcr_status_code_02),
                        (year || '03', a.mdcr_entlmt_buyin_ind_03, a.mdcr_status_code_03), (year || '04', a.mdcr_entlmt_buyin_ind_04, a.mdcr_status_code_04), (year || '05', a.mdcr_entlmt_buyin_ind_05, a.mdcr_status_code_05),
                        (year || '06', a.mdcr_entlmt_buyin_ind_06, a.mdcr_status_code_06), (year || '07', a.mdcr_entlmt_buyin_ind_07, a.mdcr_status_code_07), (year || '08', a.mdcr_entlmt_buyin_ind_08, a.mdcr_status_code_08),
                        (year || '09', a.mdcr_entlmt_buyin_ind_09, a.mdcr_status_code_09), (year || '10', a.mdcr_entlmt_buyin_ind_10, a.mdcr_status_code_10), (year || '11', a.mdcr_entlmt_buyin_ind_11, a.mdcr_status_code_11),
                        (year || '12', a.mdcr_entlmt_buyin_ind_12, a.mdcr_status_code_12))
    t(month_year_id, mcdcr_enrlmt, enrollment_status)
    join reference_tables.ref_medicare_entlmt_buyin ent 
    on ent.buyin_cd = t.mcdcr_enrlmt
    where t.enrollment_status in ('10','11','20','21','31')
),
mcrn_agg_enroll as (
    select year, bene_id, plan_type, max(month_year_id) my, count(*)
    from mcrn_enroll
    group by 1,2,3
),
mcrn_grp_enroll as (
    select *, row_number() over(partition by bene_id, year order by count desc, my desc) as my_grp
    from mcrn_agg_enroll
),
mcrn_plans as (
    select year, plan_type, count(*) plan_count
    from mcrn_grp_enroll 
    where my_grp = 1
    group by 1,2
),
dw_plans as (
    select year, plan_type, count(*) plan_count
    from dw_staging.mcrt_member_enrollment_yearly
    group by 1,2
)
select a.*, b.plan_count as src_plan_count, 
        a.plan_count - b.plan_count as plan_diff, 
        100. * abs(a.plan_count - b.plan_count) / b.plan_count as plan_diff_percent
from mcrn_plans b
full outer join dw_plans a
on a.year = b.year
and a.plan_type = b.plan_type
order by year;
'''

plan_type_counts_df = pd.read_sql(query,  con=connection)
plan_type_counts_df

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,year,plan_type,plan_count,src_plan_count,plan_diff,plan_diff_percent
0,2014,B,11841,11891.0,-50.0,0.420486
1,2014,C,1097170,NaN,NaN,NaN
2,2014,AB,2390389,3488812.0,-1098423.0,31.484156
3,2014,A,323095,321792.0,1303.0,0.404920
4,2015,AB,2373060,3600227.0,-1227167.0,34.085823
5,2015,C,1225914,NaN,NaN,NaN
6,2015,A,336500,335208.0,1292.0,0.385432
7,2015,B,13493,13532.0,-39.0,0.288206
8,2016,B,14147,14188.0,-41.0,0.288977
9,2016,AB,2379110,3706284.0,-1327174.0,35.808751


In [14]:
plan_type_counts_df.groupby('year')['plan_diff_percent'].min(), plan_type_counts_df.groupby('year')['plan_diff_percent'].max()

(year
 2014    0.404920
 2015    0.288206
 2016    0.288977
 2017    0.267926
 2018    0.481188
 2019    0.288165
 2020    0.714764
 2021    0.793615
 Name: plan_diff_percent, dtype: float64,
 year
 2014    31.484156
 2015    34.085823
 2016    35.808751
 2017    37.927757
 2018    46.948174
 2019    48.819777
 2020    50.019874
 2021    50.703767
 Name: plan_diff_percent, dtype: float64)

In [15]:
plan_type_counts_df.sort_values('plan_diff_percent')

,year,plan_type,plan_count,src_plan_count,plan_diff,plan_diff_percent
14,2017,B,14145,14183.0,-38.0,0.267926
23,2019,A,403707,402547.0,1160.0,0.288165
7,2015,B,13493,13532.0,-39.0,0.288206
8,2016,B,14147,14188.0,-41.0,0.288977
11,2016,A,349727,348431.0,1296.0,0.371953
6,2015,A,336500,335208.0,1292.0,0.385432
3,2014,A,323095,321792.0,1303.0,0.404920
0,2014,B,11841,11891.0,-50.0,0.420486
12,2017,A,367010,365301.0,1709.0,0.467833
16,2018,A,384018,382179.0,1839.0,0.481188


In [16]:
plan_types_agg_df = plan_type_counts_df.groupby('year')['plan_count', 'src_plan_count', 'plan_diff'].sum()
plan_types_agg_df['percent_diff'] = 100. * plan_types_agg_df['plan_diff'] / plan_types_agg_df['src_plan_count'] 
plan_types_agg_df

C:\Users\iperez9\AppData\Local\Temp\4\ipykernel_7812\1871058508.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  plan_types_agg_df = plan_type_counts_df.groupby('year')['plan_count', 'src_plan_count', 'plan_diff'].sum()


,plan_count,src_plan_count,plan_diff,percent_diff
year,,,,
2014,3822495,3822495.0,-1097170.0,-28.702981
2015,3948967,3948967.0,-1225914.0,-31.043916
2016,4068903,4068903.0,-1325919.0,-32.586646
2017,4194036,4194036.0,-1445103.0,-34.456142
2018,4284273,4284273.0,-1616015.0,-37.719702
2019,4419219,4419219.0,-1747669.0,-39.547010
2020,4727118,4727118.0,-1991806.0,-42.135737
2021,4818236,4818236.0,-2215618.0,-45.984007


## State

In [17]:
query = '''with mcrn_enroll as (
    select year::int, bene_id, case when state_cd is null then 'XX' else state_cd end as state
    from medicare_texas.mbsf_abcd_summary m
    left outer join reference_tables.ref_medicare_state_codes e 
     on e.medicare_state_cd = m.state_code   
),
mcrn_states as (
    select year, state, count(*) state_count
    from mcrn_enroll m
    group by 1,2
), 
dw_plans as (
    select year, case when state is null then 'XX' else state end as state, count(*) state_count
    from dw_staging.mcrt_member_enrollment_yearly
    group by 1,2
)
select a.*, b.state_count as src_state_count, 
        a.state_count - b.state_count as plan_diff, 
        100. * abs(a.state_count - b.state_count) / b.state_count as plan_diff_percent
from mcrn_states b
full join dw_plans a
on a.year = b.year
and a.state = b.state
order by year;
'''

df = pd.read_sql(query,  con=connection)
df

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,year,state,state_count,src_state_count,plan_diff,plan_diff_percent
0,2014,MS,1956,1956,0,0.000000
1,2014,PA,2277,2279,-2,0.087758
2,2014,FL,9322,9322,0,0.000000
3,2014,ND,207,207,0,0.000000
4,2014,MD,1238,1238,0,0.000000
...,...,...,...,...,...,...
427,2021,MD,1713,1713,0,0.000000
428,2021,MN,2447,2447,0,0.000000
429,2021,TX,4622637,4622841,-204,0.004413
430,2021,SD,805,805,0,0.000000


We see that there is a large difference in state values assigned using DW logic compared to raw data. Ideally the difference should show that there are more values in the raw data compared to the values in DW due, especially for state values of 'XX'. When this happens, it means that the member now has a valid state value which can be used to group these member in future projects.

In [18]:
df.sort_values('plan_diff_percent', ascending=False)

,year,state,state_count,src_state_count,plan_diff,plan_diff_percent
129,2016,XX,2099,2420,-321,13.264463
99,2015,XX,2114,2134,-20,0.937207
210,2017,XX,2217,2236,-19,0.849732
22,2014,XX,2488,2505,-17,0.678643
114,2016,AK,320,321,-1,0.311526
...,...,...,...,...,...,...
162,2017,ME,294,294,0,0.000000
161,2016,MD,1013,1013,0,0.000000
160,2016,AR,4606,4606,0,0.000000
159,2016,TN,2715,2715,0,0.000000


Should get up to 50 states plus Puerto Rico (PR), Washington D.C. (DC), Virgin Islands (VI), and unknown (XX)

In [19]:
df['state'].unique()

array(['MS', 'PA', 'FL', 'ND', 'MD', 'ME', 'IL', 'NH', 'DC', 'MT', 'NV',
       'NE', 'WA', 'LA', 'VA', 'IN', 'KS', 'HI', 'NY', 'NC', 'GA', 'CO',
       'XX', 'MN', 'PR', 'SC', 'TX', 'SD', 'MO', 'KY', 'AZ', 'RI', 'MI',
       'WY', 'DE', 'NM', 'TN', 'OR', 'VT', 'OH', 'MA', 'AK', 'ID', 'AL',
       'IA', 'UT', 'VI', 'NJ', 'AR', 'OK', 'WI', 'CT', 'WV', 'CA'],
      dtype=object)

In [20]:
df['state'].unique().shape

(54,)

In [21]:
df[df['state'] == '']

,year,state,state_count,src_state_count,plan_diff,plan_diff_percent


In [22]:
df[df['state'].isna()]

,year,state,state_count,src_state_count,plan_diff,plan_diff_percent
